# Crime rate visualisation at each SAL 

This notebook will generate a map that display the crime rate of indvidual suburb 

In [1]:
import pandas as pd
import geopandas as gpd
import folium

In [2]:
crime_df = pd.read_csv("../data/curated/crime_rate.csv")

In [3]:
### Rename the incident recorded to number of crime incident
crime_df = crime_df.rename(columns={'SAL_CODE': 'SAL_CODE21'})
crime_df

,SAL_NAME21_x,SAL_CODE21,crime_rate
0,Abbeyard,20001,0.054822
1,Abbotsford,20002,0.227735
2,Aberfeldie,20003,0.012223
3,Aberfeldy,20004,0.054822
4,Acheron,20005,0.054822
...,...,...,...
2939,Yundool,22940,0.054822
2940,Yuroke,22941,0.054822
2941,Yuulong,22942,0.054822
2942,Zeerust,22943,0.054822


### Import SAL geo data

In [4]:
# Read SAL gdf
SAL_gdf = gpd.read_file("../data/raw/victoria_region_gdf/SAL_region_gdf.geojson")
# remove null
SAL_gdf = SAL_gdf.dropna()

# Visualise it!

In [5]:
# create a JSON 
geoJSON = SAL_gdf[['SAL_CODE21', 'geometry']].to_json()

# print the first 300 chars of the json
print(geoJSON[:300])

{"type": "FeatureCollection", "features": [{"id": "0", "type": "Feature", "properties": {"SAL_CODE21": "20001"}, "geometry": {"type": "Polygon", "coordinates": [[[146.89823840500003, -37.046023841999954], [146.89946546400006, -37.048662812999964], [146.8993791040001, -37.048770832999935], [146.89915


In [6]:
m = folium.Map(location=[-36.9848, 143.3906], tiles="Stamen Terrain", zoom_start=7)

# refer to the folium documentations on more information on how to plot aggregated data.
c = folium.Choropleth(
    geo_data=geoJSON, # geoJSON 
    name='choropleth', # name of plot
    data=crime_df.reset_index(), # data source
    columns=['SAL_CODE21','crime_rate'], # the columns required
    key_on='properties.SAL_CODE21', # this is from the geoJSON's properties
    fill_color='YlOrRd', # color scheme
    nan_fill_color='black',
    legend_name='crime rate'
)

c.add_to(m)
# FILE SIZE TOO big to show 
#m


In [7]:
sorted_df = crime_df.sort_values(by='crime_rate', ascending=False)
sorted_df.head(5)

,SAL_NAME21_x,SAL_CODE21,crime_rate
1639,Melbourne,21640,1.403177
706,Dandenong,20707,0.772469
1681,Mildura,21682,0.578357
2394,Sunshine,22395,0.573904
2160,Reservoir,22161,0.544711


### Create centroid for each region

In [8]:
# (y, x) since we wanta (lat, long)
SAL_gdf['centroid'] = SAL_gdf['geometry'].apply(lambda x: (x.centroid.y, x.centroid.x))
SAL_gdf[['SAL_NAME21', 'SAL_CODE21', 'centroid']].head()

,SAL_NAME21,SAL_CODE21,centroid
0,Abbeyard,20001,"(-37.01941875010008, 146.7691121313647)"
1,Abbotsford,20002,"(-37.80458484450762, 144.99976768739876)"
2,Aberfeldie,20003,"(-37.75957575055507, 144.89740380057304)"
3,Aberfeldy,20004,"(-37.699788425368155, 146.37843544613273)"
4,Acheron,20005,"(-37.2689324082219, 145.69931893264342)"


In [9]:
# Top 5 suburb's SAL_CODE21 
values_to_find = ['21640', '20707', '21682', '22395', '22161']

# Use boolean indexing to filter rows where column 'x' contains the values from the list
filtered_rows = SAL_gdf[SAL_gdf['SAL_CODE21'].isin(values_to_find)]
filtered_rows

,SAL_NAME21,SAL_CODE21,SHAPE_Area,geometry,centroid
706,Dandenong,20707,0.001167,"POLYGON ((145.22037 -37.97068, 145.22084 -37.9...","(-37.98501237517496, 145.21187676839475)"
1639,Melbourne,21640,0.000674,"POLYGON ((144.96885 -37.80728, 144.97094 -37.8...","(-37.824913473838265, 144.97149704743745)"
1681,Mildura,21682,0.007634,"POLYGON ((142.16341 -34.17638, 142.16348 -34.1...","(-34.208414403179525, 142.12420858453862)"
2160,Reservoir,22161,0.001944,"POLYGON ((145.02209 -37.71516, 145.02201 -37.7...","(-37.7121549498847, 145.0068399920836)"
2394,Sunshine,22395,0.000484,"POLYGON ((144.83923 -37.79692, 144.84076 -37.7...","(-37.789308675988536, 144.83425757187496)"


In [10]:
for SAL_NAME21, coord in filtered_rows[['SAL_NAME21', 'centroid']].values:
    folium.Marker(
        location=coord, 
        tooltip=folium.Tooltip(SAL_NAME21, permanent=True, direction="below")
    ).add_to(m)


# # we will not diplay the html map/visual due to the file size 
# m

In [11]:
m.save('../plots/crime_rate_visual.html')
